In [58]:
import scipy as sp
import numpy as np
import pandas as pd
import utm
from statistics import mode
import torch
import os, re, glob, pyproj, math, datetime
from sys import platform
from scipy.optimize import minimize
import matplotlib.pyplot as plt

### Issues arising in likelihood optimisation

When generating parameters, the original script uses the R `optim` function to run a custom maximum likelihood estimation. This custom function takes the form of function `f(a,mx,my,wx,wy)` and runs for each speed and direction value ($s$ and $d$, respectively) within the 51 minute window.

$\sum_{i=1}^{n}{a-2 \times \log{r_r} - \left( \frac{r_r}{b}\right)^2 + m_xr_x + m_yr_y} + \log{a} - \log{b} + (1-a) \log{I_o(\sqrt{{m_x}^2 + {m_y}^2})}$

where $i,i+1,...,n$ are the values for each minute in the 51 minute window

$r_r = \sqrt{s_i\cos{d_i} - w_x}$

$b = \frac{c_v}{\Gamma{(1+1/a)}}$

$r_x = \frac{s_i\cos{d_i} - w_x}{r_r}$

$r_y = \frac{s_i\sin{d_i} - w_y}{r_r}$

$I_0$ is a zero-ordered modified Bessel function, and $\Gamma(x)$ is a gamma function.

### Issues arising in likelihood optimisation

When generating parameters, the original script uses the R `optim` function to run a custom maximum likelihood estimation. This custom function takes the form of function `f(a,mx,my,wx,wy)` and runs for each speed and direction value ($s$ and $d$, respectively) within the 51 minute window.

$\sum_{i=1}^{n}{a-2 \times \log{r_r} - \left( \frac{r_r}{b}\right)^2 + m_xr_x + m_yr_y} + \log{a} - \log{b} + (1-a) \log{I_o(\sqrt{{m_x}^2 + {m_y}^2})}$

where $i,i+1,...,n$ are the values for each minute in the 51 minute window

$r_r = \sqrt{s_i\cos{d_i} - w_x}$

$b = \frac{c_v}{\Gamma{(1+1/a)}}$

$r_x = \frac{s_i\cos{d_i} - w_x}{r_r}$

$r_y = \frac{s_i\sin{d_i} - w_y}{r_r}$

$I_0$ is a zero-ordered modified Bessel function, and $\Gamma(x)$ is a gamma function.

In [405]:
samplingInterval = 60
timeWindow = 51
cutlength = 45
cutv = 4.1667
constv = 34.7/3.6

def Likelihoodww(data1: np.ndarray,data2: np.ndarray,cv: np.ndarray): # calculate log-likelihood of the model
    def f(par):
        b = cv/sp.special.gamma(1+1/par[0])
        L = 0
        for i,j in zip(data1,data2):
            r1 = np.sqrt(pow((i*np.cos(j) - par[3]),2) + pow((i*np.sin(j) - par[4]),2))
            rx = (i*np.cos(j)-par[3])/r1
            ry = (i*np.sin(j)-par[4])/r1
            lp = (par[0]-2) * math.log(r1) - (r1/b)**par[0] + par[1]*rx + par[2]*ry + math.log(par[0]) - math.log(b) + (1-par[0])*math.log(b) - math.log(sp.special.iv(0,np.sqrt(pow(par[1],2) + pow(par[2],2)),))
            L = L+lp
        return -1/L
    return f

def Weibull_sd(a,b): # standard deviation of Weibull distribution
    return b*np.sqrt(sp.special.gamma(1+2/a) - sp.special.gamma(1+1/a)*sp.special.gamma(1+1/a))

def Weibull_mean(a,b): # mean of Weibull distribution
    return b*sp.special.gamma(1+1/a)

def Von_Mises_sd(kappa): # standard deviation of von Mises distribution
    return 1/np.sqrt(kappa)

def readAxyGPS(filename): # read in AxyTrek GPS data (txt files)
    df = pd.read_csv(filename, sep = "\t", header = None, usecols = [0,1,2,3],
    names = ['Date','Time','lat','lon'])
    df['DT'] = pd.to_datetime(df['Date'] + " " + df['Time'],format="%d/%m/%Y %H:%M:%S")
    return df

def readBIPAxy(filename): # read in AxyTrek data as formatted by BIP system
    df = pd.read_csv(filename, sep = ",", header = 0, usecols = [0,1,2], names = ['DT','lat','lon']).dropna().reset_index()
    df['DT'] = pd.to_datetime(df['DT'].str[0:-6], format = "%Y-%m-%d %H:%M:%S")
    return df

def nearest(items, pivot): # find the nearest time position
    return min(items, key=lambda x: abs(x - pivot))

def timeRescale(dat,tdiff): # calculated indeces for rescaling time (DT) for regular sampling every tdiff mins
    return dat.iloc[np.arange(0,len(dat),step=np.timedelta64(tdiff,'m')/np.timedelta64(mode(np.diff(dat['DT'])),'s')).astype(int),:].reset_index()

def distLatLon(lat,lon):
    X,Y,_,_ = utm.from_latlon(np.array(lat),np.array(lon))
    return np.append(np.nan,np.sqrt((np.diff(X))**2 + (np.diff(Y))**2))

def spTrav(DT,X,Y,threshold=0): # distance and speed from time (DT), lat, and lon with option for a threshold speed
    dist = distLatLon(X,Y)
    speed = (dist)/np.array(np.append(np.nan,np.diff(DT)/np.timedelta64(1,'s')))
    if threshold != 0:
        while np.nanmax(speed) > threshold:
            lat = lat[speed < threshold]
            lon = lon[speed < threshold]
            dist = distLatLon(X,Y)
            speed = (dist)/np.array([np.nan,np.diff(DT)/np.timedelta64(1,'s')])
    return dist, speed

def prePare(filename, convertToMin: bool = True): # prepare BIP data as per required for Goto original method. Adds columns 'dt' (elapsed time from fix to previous time point in seconds), 'dist' (distance travelled from previous point in m), 'track_speed' (in m/sec), 'track_direction' in rad
    df = readBIPAxy(filename)
    if convertToMin:
        df = timeRescale(df,1)
    df['dt'] = np.append(np.nan,(np.diff(df['DT']) / np.timedelta64(1,'s')).astype(float))
    X,Y,_,_ = utm.from_latlon(np.array(df['lat']),np.array(df['lon']))
    vg_x_obs = np.diff(X)
    vg_y_obs = np.diff(Y)
    df['dist'], df['track_speed'] = spTrav(df['DT'],df['lat'],df['lon'])
    df['track_direction'] = np.append(np.nan,[math.atan2(vg_y_obs[x],vg_x_obs[x]) for x in range(len(vg_x_obs))])
    return df

The `Likelihoodww` function takes as it's arguments two vectors: `data1`, ground speed, later referred to as $x$, `data2`, heading with east at 0 radians increasing counter-clockwise, later referred to as $y$, and `cv`, a mean air speed value estimated at 34.7 kilometers/hour. The function returns another function `f` which takes as it's argument a 5 part value `par`, whose components are assigned in order as follows: $a, m_x, m_y, w_x, w_y$. 

The variable `b` is generated as $\dfrac{cv}{\Gamma(1 + 1/a)}$ where $\Gamma(z) = \int_{0}^{\inf} t^{z - 1}e^{-t}$.

`f` then continues to run through a looped calculation for all $i$ values of $x$ and $y$. The loop is as follows and cumulatively adds to $L$ which is originally set at 0.

The loop generates the following variables:

$r_r = \sqrt{(x_i \cos(y_i) - w_x)^2 + (x_i \sin(y_i) - w_y)^2}$

$r_x = \dfrac{(x_i\cos(y_i) - wx)}{r_r}$

$r_y = \dfrac{(x_i \sin{y_i} - wy)}{r_r}$

$l_p = (a - 2) \log{r_r}) - \dfrac{r_r}{b}^a + m_x r_x + m_y r_y + \log{a} - \log{b} + (1-a) \log{b} - \log{I_n(\sqrt{(m_x^2 + m_y^2)})}$

where $I_n(x)$ is the modified Bessel function of the first kind where $I_n(x) = i^{-n}J_n(ix)$ and $J_n(x)$ refers to $J_n(x) = \dfrac{x^n}{2^n \Gamma (n+1)} {}_0F_1 \left( n + 1, -\dfrac{x^2}{4} \right)$

$L = L + lp$

In [407]:
# reread data as sample from BIP system
if platform == "darwin":
    filename = "/Users/aran/Library/CloudStorage/GoogleDrive-a-garrod@g.ecc.u-tokyo.ac.jp/My Drive/PD/Data/TestingData/SampleAxyTrek.csv"
else:
    filename = "I:/My Drive/PD/Data/TestingData/SampleAxyTrek.csv"
df = prePare(filename)

In [408]:
# # read in sample
# df = pd.read_csv("~/Desktop/Simulation_Track.csv", sep=",", header=0).dropna()
# df = df.rename(columns=lambda x: x.strip())
# # df['DT'] = pd.to_datetime(df['DT'].str[0:-6], format = "%Y-%m-%d %H:%M:%S")
# df['DT'] = pd.to_datetime(df.Date.astype(str) + " " + df.Time.astype(str), format="%d/%m/%Y %H:%M:%S")

,level_0,index,DT,lat,lon,dt,dist,track_speed,track_direction
0,0,25,2021-08-23 13:50:01,39.41653,142.27312,NaN,NaN,NaN,NaN
1,2,1550,2021-08-23 13:51:02,39.41669,142.27510,61.0,171.389536,2.809665,0.117927
2,4,3050,2021-08-23 13:52:02,39.41736,142.27547,60.0,80.902258,1.348371,1.180230
3,6,4525,2021-08-23 13:53:01,39.41845,142.27297,59.0,246.905323,4.184836,2.643619
4,8,6025,2021-08-23 13:54:01,39.42005,142.27188,60.0,200.860685,3.347678,2.071005
...,...,...,...,...,...,...,...,...,...
8282,16564,14004125,2021-08-30 01:26:05,39.40006,142.00329,61.0,242.256022,3.971410,-1.609463
8283,16566,14005600,2021-08-30 01:27:04,39.40070,142.00281,59.0,82.183063,1.392933,2.108898
8284,16568,14007125,2021-08-30 01:28:05,39.40147,141.99859,61.0,373.292817,6.119554,2.921691
8285,16570,14016950,2021-08-30 01:34:38,39.40080,141.99758,393.0,114.426821,0.291162,-2.423122


### Estimation system

Once the data are read in, the initial portion of the program deals with identifying suitable windows to run the estimation model. In the original model, these windows require 51 minutes of data. 

With a sampling frequency $f_s$ of 1 fix per minute, 51 samples would be expected, however, we assume there to be some error in the sampling interval $\epsilon$. In the original study, $\epsilon = 5$ seconds (i.e. we can expect samples to be 60 $\pm$ 5 seconds). Therefore a minimum acceptable number of samples $s_{min}$ is set, in the orginal study $s_{min} = 45$ for a 51 minute window.

This program will take a window size $w$ in minutes and from that calculate the expected number of samples $s_e = f_sw$ (e.g. 51 samples per 51 minute window for $f_s = 1$). 

Sampling interval error $\epsilon$ is assigned using a 5/60 ratio (i.e. for data with $f_s = 0.5$ or 1 fix every 30 seconds, $\epsilon = 5f_s = 2.5$ seconds). 

Finally, $s_{min} = \frac{45s_e}{51}$.

Starting from the first possible startpoint (half the window size in samples), the model then runs through the following processes:

1. Define window size.
   1. Find position of data which is 25.5 minutes after initial point.
   2. Repeat but for before initial point.
   3. Assign these positions as start and end of the window.
2. Create a new vector of track speed and direction where the speed is above the threshold of 4.1667 m/s, the sample is within 65 s of the previous sample, and direction is not equal to 100.

In [524]:
# generate a function to create windows capable of running the estimation method. Requirements are 51 minutes of data, with 75% of expected samples
def windowFit(DF,start,end,end2,cutT,cutV,cutLength):
    if (DF.DT[end] - DF.DT[start]) > np.timedelta64(cutT,'m'):
        if sum((DF.track_speed[start:end] > cutV) & (DF.dt[start:end] < cutT) & (DF.track_direction != 100)) >= cutLength:
            if (DF.DT[end2 - 1] - DF.DT[start]) < np.timedelta64(cutT,'m'):
                while (DF.DT[end2] - DF.DT[start]) < np.timedelta64(cutT,'m'):
                    end2 = end2 + 1
            return range(start,end2)
        
def findWindows(DF,windowlength=51): # calculate appropriate windows from datetimes using minimum 
    # start from minimum possible point
    fs = 1/np.abs(np.timedelta64(mode(np.diff(DF.DT)),'m')).astype(int) # in Hz
    expSamp = round(51 * fs) # expected number of samples
    cutlength = round(45/51 * fs * 51)
    error_of_sampling_interval = 5 * fs # give 5 seconds of leeway for samples to be found in
    cutt = ((60 * fs) + error_of_sampling_interval).astype(int)
    # return list(filter(None,[windowFit(DF,x,x+expSamp,x+cutlength,cutt,cutv,cutlength) for x in range(len(DF) - expSamp)]))

In [555]:
# plt.plot(df.track_speed.loc[967:1016])
1016-967

sum((df.track_speed[967:1018] > cutv))
#  & (df.dt[967:1016] < cutt) & (df.track_direction != 100))
# np.array(df.track_speed[967:1017])
# 1017-966
# df.lat[972:1023]

44

In [491]:
# if (DF.DT[end] - DF.DT[start]) > np.timedelta64(cutT,'m'):
#     if sum((DF.track_speed[start:end] > cutV) & (DF.dt[start:end] < cutT) & (DF.track_direction != 100)) >= cutLength:
#         if (DF.dt[end2 - 1] - DF.dt[start]) < np.timedelta64(cutT,'m'):
#             while (DF.dt[end2] - DF.dt[start]) < np.timedelta64(cutT,'m'):
#                 end2 = end2 + 1
(df.DT[expSamp] - df.DT[0]) > np.timedelta64(65,'m')

False

In [462]:
def windowFit(DF,tp,cutT,cutV,cutLength,windowSize):
    nxtPoint = min(range(len(DF.DT)), key=lambda i: abs(DF.DT[i]-DF.DT[tp]))
    if (nxtPoint - tp) > cutLength:
        if sum((DF.track_speed[start:end] > cutV) & (DF.dt[start:end] < cutT) & (DF.track_direction != 100)) >= cutLength:
            return np.array(range(tp,nxtPoint))[[i for i, x in enumerate((DF.track_speed[tp:nxtPoint] > cutV) & (DF.dt[tp:nxtPoint] > cutT) & (DF.track_direction[tp:nxtPoint] != 100)) if x]]

# [windowFit(df,x,cutt,cutv,cutlength,51) for x in range((len(df) - 50))]

In [464]:
for x in range((len(df)-50)):
    min(range(len(df.DT)), key=lambda i: abs(df.DT[i]-df.DT[x]))

KeyboardInterrupt: 

In [459]:
np.array(range(5,10))[[i for i, x in enumerate([True,False,False,True,True]) if x]]


# [[i for i, x in enumerate((df.track_speed[tp:nxtPoint] > cutv) & (df.dt[tp:nxtPoint] > cutt) & (df.track_direction[tp:nxtPoint] != 100)) if x]]
# 
# sum((df.track_speed[tp:nxtPoint] > cutv) & (df.dt[tp:nxtPoint] > cutt) & (df.track_direction[tp:nxtPoint] != 100))

array([5, 8, 9])

In [422]:
df.DT[761] #+ expSamp
df.DT[761 + expSamp] - df.DT[761]
df.DT[761]
# 761 + expSamp
# df.DT[761] + np.timedelta64(51,'m')



8286

In [ ]:
(DF.dt[end] - DF.dt[start]) > np.timedelta64(cutT,'m'):

# sum((df.track_speed[start:end] > cutV) & (df.deltaT[start:end] < cutT) & (df.track_direction != 100)) >= cutLength:

# (DF.dt[end2 - 1] - DF.dt[start]) < np.timedelta64(cutt,'m'):

# (DF.dt[end2] - DF.dt[start]) < np.timedelta64(cutt,'m'):
# end2 = end2 + 1

In [363]:
df2 = df.loc[(df.dt < cutt) & (~np.isnan(df.dt)) & (df.track_speed > cutv) & (df.track_direction != 100),].reset_index()
# windows = findWindows(df2,51)
# for x in range(len(df2) - cutlength):
#     windowFit(df2.DT,x,x+expSamp,cutlength,51)
# windows
# 1/np.abs(np.timedelta64(mode(np.diff(df.DT)),'m')).astype(int)
np.timedelta64(mode(np.diff(df.DT)),'s')

numpy.timedelta64(30,'s')

In [305]:
df2 = df.loc[(df.dt < cutt) & (~np.isnan(df.dt)) & (df.track_speed > cutv) & (df.track_direction != 100),].reset_index()
# df
# df2.DT.iloc[expSamp - 1] - df2.DT.iloc[0]
(df2.DT[50] - df2.DT[0]) > np.timedelta64(cutt,'m')
(df2.DT[cutlength] - df2.DT[0]) < np.timedelta64(cutt,'m')
# df2[(4753 - cutlength),]
range(len(df2) - cutlength)

range(0, 4708)

Run through each datetime value from 1 to end - cutlength. First test if `datetime[i]:datetime[i+51]` is over 51 mins. If yes, then find if `datetime[i]:datetime[i+cutlength]` is over 51 mins. If no, set up a while loop to identify at what index the value is over 51 minutes.

## Process:

At this stage, the model starts to run through a variety of 'initial headings', set as each integer between -3 and 3. For each initial heading, the following processes are run:

1. Create variable `inita` as a random variable generated from a normal distribution with mean 12.5 and standard deviation 5. `inita` must be greater than 5.
2. Calculate the mean heading from all headings within the window that passed the above requirements.
3. The sum of the initial heading and the mean heading is determined, and using these data, the following variables are estimated:
   1. `kappa`: the concentration parameter for a von Mises distribution
   2. `mux` and `muy`: the x and y components of `kappa`
   3. `wx` and `wy`: the x and y components of wind (the track vector - the heading)
4. The `inita`, `mux`, `muy`, `wx`, and `wy` variables are then optimised using log-likelihood and track speed and direction data alongside a constant assumed mean air speed (34.7 m/s).
5. The standard deviation of the heading vector perpendicular to the mean direction `yoko` and the standard deviation of the heading vector along the mean direction `tate` are calculated.

If convergence is not reached but `tate` can be calculated, the process is repeated until convergence is reached.


1. First identify a suitable window.
   1. Determine the start and end such that they are greater than `windwidthsec` apart.
   2. Extract only data where the speed is greater than `cutv`, time difference is less than `cutt`, and the heading is not equal to 100 (when direction cannot be calculated i.e. track speed = 0).
2. If the resultant window is greater in length than `cutlength`, proceed to maximum likelihood modelling.

In [171]:
# remove low speed values and where time differences are too long
dfRm = df[(df['track_speed'] > cutv) & (df['dt'] < cutt)].reset_index()
# find suitable windows
windows = findWindows(dfRm['DT'],51)
rrow = dfRm.track_speed
drow = dfRm.track_direction

In R, the function `A1inv` returns a value `k` from input argument `r` such that

$A1inv(k) = A1(r)$

where 

$A1(r) = \frac{I_1(\kappa)}{I_0(\kappa)}$

where $I_1$ and $I_0$ are the first and zeroth order Bessel functions, respectively.

However, Python does not have such a function, but the `circular` package has a [GitHub page](https://github.com/cran/circular) documenting the functions, including `A1inv` and so we can now simply implement this within Python as a new function.

In [ ]:
def A1inv(x):
    if ((0 <= x) * (x < 0.53)):
        return 2 * x + pow(x,3) + (5 * pow(x,5))/6
    else:
        if (x < 0.85):
            return -0.4 + 1.39 * x + 0.43/(1-x)
        else:
            return 1/(pow(x,3) - 4 * pow(x,2) + 3 * x)

The initial method ran through a loop groung through every potential fitting time window. Instead, I think it would be best to optimise the process by first identifying usable windows then running through all those. This would remove the processing time for all none-fitting windows.

Suitable windows are ones which pass the following requirements:
1. Window length is greater or equal to minimum window size in seconds
2. Windows contain sufficient number of data points (75% of expected samples)
3. Data in window must be collected at speeds greater than the minimum ground speed (4.1667 m/s)

To speed up calculation, create two versions, one which generates winds every hour or so. Wind should not change so frequently and this may be optimal for more users. Also very useful for trial users.

In [173]:
# windProcess(df.track_speed[965:1014],df.track_direction[965:1014],range(-3,3),constv)

# set initial value for max likelihood and best answer
max_like = np.nan
answ_best = np.nan
# loop through each hd_try value
for heading in range(-3,3):
    # create initial parameters to maximise likelihood function
    initPars = initHead(heading,df.track_speed[965:1014],df.track_direction[965:1014],constv)
    # run first optimisation
    answ = windOptims(df.track_speed[965:1014],df.track_direction[965:1014],constv,initPars)
    # generate sd of heading vector perp to mean direction and along mean direction
    yoko,tate = yokoTate(answ,constv)
    # # repeat MLE to ensure optimisation convergence
    if answ.success:
        answ = ensureOptimConv(answ,df.track_speed[965:1014],df.track_direction[965:1014],constv)
        yoko,tate = yokoTate(answ,constv)
    # if successfully converged
    if answ.success:
        nr, nd = headSpDir(df.track_speed[965:1014],df.track_direction[965:1014],answ.x[3],answ.x[4])
        # define mean heading
        meangd = np.arctan2(np.mean(np.sin(df.track_direction[965:1014])),np.mean(np.cos(df.track_direction[965:1014])))
        # test goodness of fit
        if np.isnan(max_like) & (corrTests(nr,nd,answ,yoko,tate,constv,meangd) == 1):
            max_like = answ.fun
            answ_best = answ
        else:
            answ_best = np.nan
        if (answ.fun > max_like) & (corrTests(nr,nd,answ,yoko,tate,constv,meangd) == 1):
            max_like = answ.fun
            answ_best = answ
        else:
            answ_best = np.nan

In [179]:
# for x in windows[0:10]:
#     print(windProcess(df.track_speed[x],df.track_direction[x],range(-3,3),constv))
windProcess(df.track_speed[966:1015],df.track_direction[x],range(-3,3),constv)

(nan, nan)

In [121]:
# answ.x = [6.033869,25.836523,-6.398774,-6.269189,11.549025]
# yoko,tate = yokoTate(answ,constv)
# Von_Mises_sd(np.sqrt(answ.x[2]*answ.x[2]+answ.x[1]*answ.x[1]))*Weibull_mean(answ.x[0],constv/sp.special.gamma(1+1/answ.x[0]))
# speed of heading vector
nrp = sp.stats.kstest(nr,'weibull_min',args=[answ.x[0],constv/sp.special.gamma(1+1/answ.x[0])])
# # direction of heading vector
mu = math.atan2(answ.x[2],answ.x[1])
kappa = np.sqrt(pow(answ.x[1],2) + answ.x[2]*answ.x[2])
ndp = sp.stats.kstest(nd,'vonmises',args=[kappa,mu])
# # correlation test between direction and speed of heading vector
cnrnd = sp.stats.pearsonr(nr,nd)
# # describe conditions required
# cond1 = (yoko/tate) > 1
# cond2 = (np.cos(meangd)*np.cos(mu) + np.sin(meangd)*np.sin(mu)) > 0
# cond3 = (nrp.pvalue > 0.05) * (ndp.pvalue > 0.05) * (cnrnd.pvalue > 0.05)
cnrnd

PearsonRResult(statistic=-0.230828842500166, pvalue=0.1031706811005095)